# EX10-GRAPH: Spark GraphX (Scala API)

Your assignment: complete the `TODO`'s and include also the **output of each cell**.

### Step 1: Almond configuration for Spark/Scala kernels

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import $ivy.`org.apache.spark::spark-graphx:3.5.0`

import $ivy.$
import $ivy.$

### Step 2: Start Spark session (notice that this is specific for Almond Notebooks)

In [3]:
import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

spark.sparkContext.setLogLevel("ERROR")

Getting spark JARs

Creating SparkSession

Spark UI

import org.apache.spark.sql._
spark: SparkSession = org.apache.spark.sql.SparkSession@1708696f

### Step 3: Spark Official Doc Example

In [4]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

class VertexProperty()
case class UserProperty(val name: String) extends VertexProperty
case class ProductProperty(val name: String, val price: Double) extends VertexProperty

val sc = spark.sparkContext

val users: RDD[(VertexId, (String, String))] =
  sc.parallelize(Seq((3L, ("rxin", "student")), (7L, ("jgonzal", "postdoc")),
                       (5L, ("franklin", "prof")), (2L, ("istoica", "prof"))))
val relationships: RDD[Edge[String]] =
  sc.parallelize(Seq(Edge(3L, 7L, "collab"),    Edge(5L, 3L, "advisor"),
                       Edge(2L, 5L, "colleague"), Edge(5L, 7L, "pi")))
val defaultUser = ("John Doe", "Missing")
val graph = Graph(users, relationships, defaultUser)

graph.vertices.collect()

import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
defined class VertexProperty
defined class UserProperty
defined class ProductProperty
sc: SparkContext = org.apache.spark.SparkContext@76ed26f
users: RDD[(VertexId, (String, String))] = ParallelCollectionRDD[0] at parallelize at cmd4.sc:14
relationships: RDD[Edge[String]] = ParallelCollectionRDD[1] at parallelize at cmd4.sc:18
defaultUser: (String, String) = ("John Doe", "Missing")
graph: Graph[(String, String), String] = org.apache.spark.graphx.impl.GraphImpl@4101e0f8
res4_11: Array[(VertexId, (String, String))] = Array(
  (2L, ("istoica", "prof")),
  (3L, ("rxin", "student")),
  (5L, ("franklin", "prof")),
  (7L, ("jgonzal", "postdoc"))
)

### Step 4: Try out other constructs and capabilities of GraphX from the official doc (`TODO`)

Include other commands, see what it does. Try out the Pregel API, etc.

[GraphX Documentation](https://spark.apache.org/docs/latest/graphx-programming-guide.html)


In [5]:
graph.edges.collect()

res5: Array[Edge[String]] = Array(
  Edge(3L, 7L, "collab"),
  Edge(5L, 3L, "advisor"),
  Edge(2L, 5L, "colleague"),
  Edge(5L, 7L, "pi")
)

In [6]:
val degrees = graph.degrees
degrees.collect()

degrees: VertexRDD[Int] = VertexRDDImpl[18] at RDD at VertexRDD.scala:57
res6_1: Array[(VertexId, Int)] = Array((2L, 1), (3L, 2), (5L, 3), (7L, 2))

In [7]:
graph.edges.filter(_.attr.length > 5).map(_.attr).collect()

res7: Array[String] = Array("collab", "advisor", "colleague")

In [8]:
val profSubgraph = graph.subgraph(vpred = (id, attr) => attr._2 == "prof")
profSubgraph.vertices.collect()

profSubgraph: Graph[(String, String), String] = org.apache.spark.graphx.impl.GraphImpl@8ec17aa
res8_1: Array[(VertexId, (String, String))] = Array(
  (2L, ("istoica", "prof")),
  (5L, ("franklin", "prof"))
)

In [9]:
val ranks = graph.pageRank(0.0001).vertices
ranks.join(users).map {
  case (id, (rank, (name, title))) => (id, (name, title))
}.collect()

ranks: VertexRDD[Double] = VertexRDDImpl[118] at RDD at VertexRDD.scala:57
res9_1: Array[(VertexId, (String, String))] = Array(
  (2L, ("istoica", "prof")),
  (3L, ("rxin", "student")),
  (5L, ("franklin", "prof")),
  (7L, ("jgonzal", "postdoc"))
)

### Step 5: Modeling another graph

Exercise 3 (data models) asked you to build a graph from nobel data. The challenge is to use everything you have learned so far (Spark core, SparkSQL) to read that data and build the same graph using Spark/GraphX data type. Things to remember: nobel data is available in JSON format, back then we used Python lib networkx to model that graph. Now you must do this in Spark, back to back.


In [18]:
import org.apache.spark.sql._ 
import org.apache.spark.sql.functions._ 
import org.apache.spark.graphx._ 

val raw = spark.read
  .option("multiline", "true")
  .json("data/laureate.json")

raw.printSchema()   
raw.select("laureates.id","laureates.born","laureates.died").show(5,false)


root
 |-- laureates: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- born: string (nullable = true)
 |    |    |-- bornCity: string (nullable = true)
 |    |    |-- bornCountry: string (nullable = true)
 |    |    |-- bornCountryCode: string (nullable = true)
 |    |    |-- died: string (nullable = true)
 |    |    |-- diedCity: string (nullable = true)
 |    |    |-- diedCountry: string (nullable = true)
 |    |    |-- diedCountryCode: string (nullable = true)
 |    |    |-- firstname: string (nullable = true)
 |    |    |-- gender: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- prizes: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- affiliations: array (nullable = true)
 |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |-- category: string (nullable = true)
 |    |    |    |    |-- motivation: string (nullable = 

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.graphx._
raw: DataFrame = [laureates: array<struct<born:string,bornCity:string,bornCountry:string,bornCountryCode:string,died:string,diedCity:string,diedCountry:string,diedCountryCode:string,firstname:string,gender:string,id:string,prizes:array<struct<affiliations:array<string>,category:string,motivation:string,overallMotivation:string,share:string,year:string>>,surname:string>>]

In [19]:
val raw = spark.read
  .option("multiline", "true")
  .json("data/laureate.json")

import spark.implicits._
val exploded = raw
  .select(explode($"laureates").as("l"))

val df = exploded.select(
  $"l.id".cast("long").as("id"),
  concat_ws(" ", $"l.firstname", $"l.surname").as("name"),
  $"l.born".as("born"),
  $"l.died".as("died")
)

df.show(5,false)
df.printSchema()


+---+----------------------+----------+----------+
|id |name                  |born      |died      |
+---+----------------------+----------+----------+
|1  |Wilhelm Conrad Röntgen|1845-03-27|1923-02-10|
|2  |Hendrik A. Lorentz    |1853-07-18|1928-02-04|
|3  |Pieter Zeeman         |1865-05-25|1943-10-09|
|4  |Henri Becquerel       |1852-12-15|1908-08-25|
|5  |Pierre Curie          |1859-05-15|1906-04-19|
+---+----------------------+----------+----------+
only showing top 5 rows

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = false)
 |-- born: string (nullable = true)
 |-- died: string (nullable = true)



raw: DataFrame = [laureates: array<struct<born:string,bornCity:string,bornCountry:string,bornCountryCode:string,died:string,diedCity:string,diedCountry:string,diedCountryCode:string,firstname:string,gender:string,id:string,prizes:array<struct<affiliations:array<string>,category:string,motivation:string,overallMotivation:string,share:string,year:string>>,surname:string>>]
import spark.implicits._
exploded: DataFrame = [l: struct<born: string, bornCity: string ... 11 more fields>]
df: DataFrame = [id: bigint, name: string ... 2 more fields]

In [20]:
import org.apache.spark.sql._ 
import org.apache.spark.sql.functions._ 
import org.apache.spark.graphx._
import spark.implicits._

val raw = spark.read
  .option("multiline", "true")
  .json("data/laureate.json")

val df = raw
  .select(explode($"laureates").as("l"))
  .select(
    $"l.id".cast("long").as("id"),
    concat_ws(" ", $"l.firstname", $"l.surname").as("name"),
    $"l.prizes".as("prizes")
  )


import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.graphx._
import spark.implicits._
raw: DataFrame = [laureates: array<struct<born:string,bornCity:string,bornCountry:string,bornCountryCode:string,died:string,diedCity:string,diedCountry:string,diedCountryCode:string,firstname:string,gender:string,id:string,prizes:array<struct<affiliations:array<string>,category:string,motivation:string,overallMotivation:string,share:string,year:string>>,surname:string>>]
df: DataFrame = [id: bigint, name: string ... 1 more field]

In [21]:
val vertices: RDD[(VertexId, String)] =
  df.select($"id", $"name").as[(Long, String)].rdd

val laureatePrizes = df
  .select($"id", explode($"prizes").as("p"))
  .select($"id", concat_ws("-", $"p.year", $"p.category").as("prize"))
  .as[(Long, String)]
  .rdd

val edges: RDD[Edge[String]] =
  laureatePrizes
    .groupBy(_._2)
    .flatMap { case (prize, seq) =>
      val ids = seq.map(_._1).toArray
      for {
        i <- 0 until ids.length
        j <- (i+1) until ids.length
      } yield Edge(ids(i), ids(j), prize)
    }


vertices: RDD[(VertexId, String)] = MapPartitionsRDD[82] at rdd at cmd21.sc:2
laureatePrizes: RDD[(Long, String)] = MapPartitionsRDD[88] at rdd at cmd21.sc:7
edges: RDD[Edge[String]] = MapPartitionsRDD[91] at flatMap at cmd21.sc:13

In [22]:
val defaultName = "desconhecido"
val graph = Graph(vertices, edges, defaultName)

// exemplo de saída
graph.triplets.take(10).foreach { t =>
  println(s"${t.srcAttr} ↔[${t.attr}]↔ ${t.dstAttr}")
}


Hendrik A. Lorentz ↔[1902-physics]↔ Pieter Zeeman
Henri Becquerel ↔[1903-physics]↔ Pierre Curie
Henri Becquerel ↔[1903-physics]↔ Marie Curie
Pierre Curie ↔[1903-physics]↔ Marie Curie
Guglielmo Marconi ↔[1909-physics]↔ Ferdinand Braun
William Bragg ↔[1915-physics]↔ Lawrence Bragg
James Franck ↔[1925-physics]↔ Gustav Hertz
Arthur H. Compton ↔[1927-physics]↔ C.T.R. Wilson
Erwin Schrödinger ↔[1933-physics]↔ Paul A.M. Dirac
Victor F. Hess ↔[1936-physics]↔ Carl D. Anderson


defaultName: String = "desconhecido"
graph: Graph[String, String] = org.apache.spark.graphx.impl.GraphImpl@7dc80a4a